In [29]:
## preprocessing.py imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import set_config
import joblib
import xgboost


#custom 
from smackbang.preprocess import DateFormatter, DateEncoder, TimeFeaturesEncoder, haversine_vectorized, DistanceTransformer, duration_process, set_preproc_pipeline


set_config(display='diagram')



/home/lee/.pyenv/versions/3.8.12/envs/smackbang/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
import requests
import pandas as pd
from requests.structures import CaseInsensitiveDict

url = "https://smackbang-image-w76hg6ifha-ew.a.run.app/matches"

headers = CaseInsensitiveDict()
headers["accept"] = "application/json"

origin_one = 'NRT'
origin_two = 'SYD'
departure_date = '01/04/2022'
continent='AS'

query_string = {'origin_one':origin_one, 'origin_two':origin_two, 'departure_date':departure_date, 
                'continent':continent, 'return_date':'', 'currency':'USD'}

result = requests.get(url, headers=headers, params=query_string).json()
                
matches_df = pd.DataFrame(result)

In [41]:
matches_df

,price_Sydney,price_Tokyo,duration_Sydney,duration_Tokyo,total_stops_Sydney,total_stops_Tokyo,distance_Sydney,distance_Tokyo,local_departure_Sydney,local_departure_Tokyo,local_arrival_Sydney,local_arrival_Tokyo,deep_link_Sydney,deep_link_Tokyo,combined_price_,combined_duration_,lat_,lon_
Bangkok,351,391,9.83,6.50,0,0,7523.15,4653.65,2022-04-01T06:35:00.000Z,2022-04-01T12:00:00.000Z,2022-04-01T16:40:00.000Z,2022-04-01T16:30:00.000Z,https://www.kiwi.com/deep?from=SYD&to=BKK&flig...,https://www.kiwi.com/deep?from=NRT&to=BKK&flig...,742,16.33,13.681100,100.747002
Hanoi,377,391,18.00,6.33,1,0,7798.15,3723.95,2022-04-01T13:25:00.000Z,2022-04-01T09:15:00.000Z,2022-04-02T11:25:00.000Z,2022-04-01T13:35:00.000Z,https://www.kiwi.com/deep?from=SYD&to=HAN&flig...,https://www.kiwi.com/deep?from=NRT&to=HAN&flig...,768,24.33,21.221201,105.806999
Taipei,376,419,19.33,3.67,1,0,7293.46,2183.46,2022-04-01T13:25:00.000Z,2022-04-01T13:50:00.000Z,2022-04-02T05:45:00.000Z,2022-04-01T16:30:00.000Z,https://www.kiwi.com/deep?from=SYD&to=TPE&flig...,https://www.kiwi.com/deep?from=NRT&to=TPE&flig...,795,23.00,25.077700,121.233002
Osaka,741,80,12.33,1.50,1,0,7824.93,462.63,2022-04-01T21:45:00.000Z,2022-04-01T07:00:00.000Z,2022-04-02T08:05:00.000Z,2022-04-01T08:35:00.000Z,https://www.kiwi.com/deep?from=SYD&to=ITM&flig...,https://www.kiwi.com/deep?from=NRT&to=ITM&flig...,821,13.83,34.785500,135.438004
Kuala Lumpur,291,533,8.83,14.67,0,1,6591.75,5414.54,2022-04-01T06:25:00.000Z,2022-04-01T19:00:00.000Z,2022-04-01T15:35:00.000Z,2022-04-02T08:40:00.000Z,https://www.kiwi.com/deep?from=SYD&to=KUL&flig...,https://www.kiwi.com/deep?from=NRT&to=KUL&flig...,824,23.50,2.745580,101.709999
Singapore,270,570,8.25,7.25,0,0,6301.43,5363.07,2022-04-01T08:15:00.000Z,2022-04-01T18:15:00.000Z,2022-04-01T16:50:00.000Z,2022-04-02T00:35:00.000Z,https://www.kiwi.com/deep?from=SYD&to=SIN&flig...,https://www.kiwi.com/deep?from=NRT&to=SIN&flig...,840,15.50,1.350190,103.994003
Fukuoka,738,103,13.08,2.08,1,0,7823.25,940.88,2022-04-01T21:45:00.000Z,2022-04-01T07:10:00.000Z,2022-04-02T08:50:00.000Z,2022-04-01T09:15:00.000Z,https://www.kiwi.com/deep?from=SYD&to=FUK&flig...,https://www.kiwi.com/deep?from=NRT&to=FUK&flig...,841,15.16,33.585899,130.451004
Manila,479,385,8.33,4.75,0,0,6269.74,3054.20,2022-04-01T11:15:00.000Z,2022-04-01T13:10:00.000Z,2022-04-01T16:50:00.000Z,2022-04-01T17:30:00.000Z,https://www.kiwi.com/deep?from=SYD&to=MNL&flig...,https://www.kiwi.com/deep?from=NRT&to=MNL&flig...,864,13.08,14.508600,121.019997
Ho Chi Minh City,447,445,8.92,7.00,0,0,6853.68,4388.16,2022-04-01T07:00:00.000Z,2022-04-01T08:55:00.000Z,2022-04-01T16:10:00.000Z,2022-04-01T13:55:00.000Z,https://www.kiwi.com/deep?from=SYD&to=SGN&flig...,https://www.kiwi.com/deep?from=NRT&to=SGN&flig...,892,15.92,10.818800,106.652000
Da Nang,528,421,27.75,9.83,1,1,7190.96,3869.90,2022-04-01T16:00:00.000Z,2022-04-01T09:15:00.000Z,2022-04-02T15:45:00.000Z,2022-04-01T17:05:00.000Z,https://www.kiwi.com/deep?from=SYD&to=DAD&flig...,https://www.kiwi.com/deep?from=NRT&to=DAD&flig...,949,37.58,16.043900,108.198998


In [4]:
city_one = 'Sydney'
city_two = 'Tokyo'

In [5]:
# preprocessing matches_df for fare prediction model
# need to split the matches_df into two seperate dataframes then concat along the rows.
df1 = matches_df.iloc[:,2:12:2]

# convert local dearture to datetime
df1[f'local_departure_{city_one}'] = pd.to_datetime(df1[f'local_departure_{city_one}'])

# convert local arrival to datetime
df1[f'local_arrival_{city_one}'] = pd.to_datetime(df1[f'local_arrival_{city_one}'])

# create and format Date_of_Journey column based on local_departure
df1['Date_of_Journey'] = df1[f'local_departure_{city_one}'].dt.date[0].strftime("%d/%m/%Y")

# create and format Dep_Time column based on local_departure
df1["Dep_Time"] = df1[f"local_departure_{city_one}"].apply(lambda x: x.strftime("%H:%M"))

# create and format Arrival_time column based on local_arrival
df1["Arrival_Time"] = df1[f"local_arrival_{city_one}"].apply(lambda x: x.strftime("%H:%M"))

# create and format Duration 
df1['Duration'] = df1[f"duration_{city_one}"].apply(lambda x: f"{int(x)}h {(round(x*60)%60)}m")

# Convert Total_Stops_city_one to strings
df1[f'total_stops_{city_one}'] = df1[f'total_stops_{city_one}'].apply(str)

# create Source Column based on city_one
df1['Source'] = city_one

# create placeholder column for Additional_Info. This column will be dropped during preproc pipeline
df1['Additional_Info'] = "No info"

# create placeholder column for Airline. This column will be dropped during preproc pipeline
df1["Airline"] = "Multiple Carriers"

airports_df = pd.read_csv('../data/airport_codes.csv')

#creating empty lat,long columns type float64
df1['origin_one_latitude'] = np.nan
df1['origin_one_longitude'] = np.nan

# grabbing lat,lon from airports df
for city in list(df1.index):
    if city in list(airports_df['city']):
        df1.loc[city,'origin_one_latitude']=list(airports_df[airports_df['city']==city]['lat'])[0]
        df1.loc[city,'origin_one_longitude']=list(airports_df[airports_df['city']==city]['lon'])[0]

#creating empty lat,long columns type float64
df1['origin_one_latitude'] = np.nan
df1['origin_one_longitude'] = np.nan

# grabbing lat,lon from airports df
for city in list(df1.index):
    if city in list(airports_df['city']):
        df1.loc[city,'origin_one_latitude']=list(airports_df[airports_df['city']==city]['lat'])[0]
        df1.loc[city,'origin_one_longitude']=list(airports_df[airports_df['city']==city]['lon'])[0]

df1['origin_two_latitude'] = np.nan
df1['origin_two_longitude'] = np.nan

df1['origin_two_latitude'] = list(airports_df[airports_df['city']==city_one]['lat'])[0]
df1['origin_two_longitude'] = list(airports_df[airports_df['city']==city_one]['lon'])[0]

# creating a new columns from the index which will be used for the Destination column
df1 = df1.reset_index()

# rename index column to Destination
df1.rename(columns={'index':'Destination',f'total_stops_{city_one}':'Total_Stops'},inplace=True)

# create placeholder column for Route. This column will be dropped during preproc pipeline
df1['Route'] = df1["Source"] + "-" + df1["Destination"]

df1_formatted = df1[['Airline','Date_of_Journey','Source','Destination',
                     'Route','Dep_Time','Arrival_Time','Duration','Total_Stops',
                     'Additional_Info','origin_one_latitude','origin_one_longitude',
                    'origin_two_latitude','origin_two_longitude']]

In [48]:
# preprocessing matches_df for fare prediction model
# need to split the matches_df into two seperate dataframes then concat along the rows.
df2 = matches_df.iloc[:,3:12:2]

# convert local dearture to datetime
df2[f'local_departure_{city_two}'] = pd.to_datetime(df2[f'local_departure_{city_two}'])

# convert local arrival to datetime
df2[f'local_arrival_{city_two}'] = pd.to_datetime(df2[f'local_arrival_{city_two}'])

# create and format Date_of_Journey column based on local_departure
df2['Date_of_Journey'] = df2[f'local_departure_{city_two}'].dt.date[0].strftime("%d/%m/%Y")

# create and format Dep_Time column based on local_departure
df2["Dep_Time"] = df2[f"local_departure_{city_two}"].apply(lambda x: x.strftime("%H:%M"))

# create and format Arrival_time column based on local_arrival
df2["Arrival_Time"] = df2[f"local_arrival_{city_two}"].apply(lambda x: x.strftime("%H:%M"))

# create and format Duration 
df2['Duration'] = df2[f"duration_{city_two}"].apply(lambda x: f"{int(x)}h {(round(x*60)%60)}m")

# Convert Total_Stops_city_one to strings
df2[f'total_stops_{city_two}'] = df2[f'total_stops_{city_two}'].apply(str)

# create Source Column based on city_one
df2['Source'] = city_two

# create placeholder column for Additional_Info. This column will be dropped during preproc pipeline
df2['Additional_Info'] = "No info"

# create placeholder column for Airline. This column will be dropped during preproc pipeline
df2["Airline"] = "Multiple Carriers"

airports_df = pd.read_csv('../data/airport_codes.csv')

#creating empty lat,long columns type float64
df2['origin_one_latitude'] = np.nan
df2['origin_one_longitude'] = np.nan

# grabbing lat,lon from airports df
for city in list(df2.index):
    if city in list(airports_df['city']):
        df2.loc[city,'origin_one_latitude']=list(airports_df[airports_df['city']==city]['lat'])[0]
        df2.loc[city,'origin_one_longitude']=list(airports_df[airports_df['city']==city]['lon'])[0]

#creating empty lat,long columns type float64
df2['origin_one_latitude'] = np.nan
df2['origin_one_longitude'] = np.nan

# grabbing lat,lon from airports df
for city in list(df2.index):
    if city in list(airports_df['city']):
        df2.loc[city,'origin_one_latitude']=list(airports_df[airports_df['city']==city]['lat'])[0]
        df2.loc[city,'origin_one_longitude']=list(airports_df[airports_df['city']==city]['lon'])[0]

df2['origin_two_latitude'] = np.nan
df2['origin_two_longitude'] = np.nan

df2['origin_two_latitude'] = list(airports_df[airports_df['city']==city_one]['lat'])[0]
df2['origin_two_longitude'] = list(airports_df[airports_df['city']==city_one]['lon'])[0]

# creating a new columns from the index which will be used for the Destination column
df2 = df2.reset_index()

# rename index column to Destination
df2.rename(columns={'index':'Destination',f'total_stops_{city_two}':'Total_Stops'},inplace=True)

# create placeholder column for Route. This column will be dropped during preproc pipeline
df2['Route'] = df2["Source"] + "-" + df2["Destination"]

df2_formatted = df2[['Airline','Date_of_Journey','Source','Destination',
                     'Route','Dep_Time','Arrival_Time','Duration','Total_Stops',
                     'Additional_Info','origin_one_latitude','origin_one_longitude',
                    'origin_two_latitude','origin_two_longitude']]

In [49]:
df2_formatted

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,origin_one_latitude,origin_one_longitude,origin_two_latitude,origin_two_longitude
0,Multiple Carriers,01/04/2022,Tokyo,Bangkok,Tokyo-Bangkok,12:00,16:30,6h 30m,0,No info,13.681100,100.747002,-33.946098,151.177002
1,Multiple Carriers,01/04/2022,Tokyo,Hanoi,Tokyo-Hanoi,09:15,13:35,6h 20m,0,No info,21.221201,105.806999,-33.946098,151.177002
2,Multiple Carriers,01/04/2022,Tokyo,Taipei,Tokyo-Taipei,13:50,16:30,3h 40m,0,No info,25.077700,121.233002,-33.946098,151.177002
3,Multiple Carriers,01/04/2022,Tokyo,Osaka,Tokyo-Osaka,07:00,08:35,1h 30m,0,No info,34.785500,135.438004,-33.946098,151.177002
4,Multiple Carriers,01/04/2022,Tokyo,Kuala Lumpur,Tokyo-Kuala Lumpur,19:00,08:40,14h 40m,1,No info,2.745580,101.709999,-33.946098,151.177002
5,Multiple Carriers,01/04/2022,Tokyo,Singapore,Tokyo-Singapore,18:15,00:35,7h 15m,0,No info,1.350190,103.994003,-33.946098,151.177002
6,Multiple Carriers,01/04/2022,Tokyo,Fukuoka,Tokyo-Fukuoka,07:10,09:15,2h 5m,0,No info,33.585899,130.451004,-33.946098,151.177002
7,Multiple Carriers,01/04/2022,Tokyo,Manila,Tokyo-Manila,13:10,17:30,4h 45m,0,No info,14.508600,121.019997,-33.946098,151.177002
8,Multiple Carriers,01/04/2022,Tokyo,Ho Chi Minh City,Tokyo-Ho Chi Minh City,08:55,13:55,7h 0m,0,No info,10.818800,106.652000,-33.946098,151.177002
9,Multiple Carriers,01/04/2022,Tokyo,Da Nang,Tokyo-Da Nang,09:15,17:05,9h 50m,1,No info,16.043900,108.198998,-33.946098,151.177002


In [59]:
df3_formatted = pd.concat([df1_formatted, df2_formatted],ignore_index=True)

In [33]:
pipeline = joblib.load('../model.joblib')

In [62]:
fare_pred1 = pipeline.predict(df1_formatted)

['9h 50m' '18h 0m' '19h 20m' '12h 20m' '8h 50m' '8h 15m' '13h 5m' '8h 20m'
 '8h 55m' '27h 45m' '11h 0m' '23h 45m' '34h 5m' '15h 50m' '42h 20m'
 '36h 5m' '30h 40m']


In [63]:
fare_pred2 = pipeline.predict(df2_formatted)

['6h 30m' '6h 20m' '3h 40m' '1h 30m' '14h 40m' '7h 15m' '2h 5m' '4h 45m'
 '7h 0m' '9h 50m' '7h 45m' '2h 30m' '21h 15m' '11h 35m' '11h 45m' '15h 0m'
 '18h 35m']


In [67]:
result1 = {'city_one':list(fare_pred1)}

In [68]:
pd.DataFrame(result1)

,city_one
0,6315.384766
1,7341.482910
2,9992.184570
3,8744.046875
4,6923.974121
5,10020.401367
6,10656.093750
7,10331.476562
8,8455.984375
9,6735.811523


In [71]:
result2 = {'city_two':list(fare_pred2)}

In [72]:
pd.DataFrame(result2)

,city_two
0,9675.667969
1,4737.806152
2,5332.179688
3,4476.482422
4,8551.655273
5,14044.267578
6,4853.105469
7,5949.681641
8,6410.751465
9,7918.945312
